In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from Utils import load
from Utils import generator
from Utils import metrics
from train import *
from prune import *
from Layers import layers

In [2]:
from torch.nn import functional as F
import torch.nn as nn
def fc(input_shape, nonlinearity=nn.ReLU()):
      size = np.prod(input_shape)

      # Linear feature extractor
      modules = [nn.Flatten()]
      modules.append(layers.Linear(size, 5000))
      modules.append(nonlinearity)
      modules.append(layers.Linear(5000, 900))
      modules.append(nonlinearity)
      modules.append(layers.Linear(900, 400))
      modules.append(nonlinearity)
      modules.append(layers.Linear(400, 100))
      modules.append(nonlinearity)        
      modules.append(layers.Linear(100, 30))
      modules.append(nonlinearity)
      modules.append(layers.Linear(30, 1))

      model = nn.Sequential(*modules)

      return model

In [ ]:

from data import *
from models import *
from utils import *
from sklearn.model_selection import KFold
import os, shutil, pickle
ctx = mx.gpu(0) if mx.context.num_gpus() > 0 else mx.cpu(0)

loss_before_prune=[]
loss_after_prune=[]
loss_prune_posttrain=[]
NUM_PARA=[]
for datasetindex in range(10):#[0,1,4,5,6,7,8,9]:
    dataset=str(datasetindex)+'.csv'
    X, y= get_data(dataset)

    np.random.seed(0)    
    kf = KFold(n_splits=5,random_state=0,shuffle=True)
    kf.get_n_splits(X)
    seed=0#[0,1,2,3,4]
    chosenarmsList=[]
    for train_index, test_index in kf.split(X):
        X_tr, X_te = X[train_index], X[test_index]
        y_tr, y_te = y[train_index], y[test_index]
        X_test=nd.array(X_te).as_in_context(ctx)  # Fix test data for all seeds
        y_test=nd.array(y_te).as_in_context(ctx)
        factor=np.max(y_te)-np.min(y_te) #normalize RMSE
        print(factor)
        #X_tr, X_te, y_tr, y_te = get_data(0.2,0)
        #selected_interaction = detectNID(X_tr,y_tr,X_te,y_te,test_size,seed)
        #index_Subsets=get_interaction_index(selected_interaction)
        
        N=X_tr.shape[0]
        p=X_tr.shape[1]
        batch_size=500
        n_epochs=300
        if N<250:
            batch_size=50
        X_train=nd.array(X_tr).as_in_context(ctx)
        y_train=nd.array(y_tr).as_in_context(ctx)
        train_dataset = ArrayDataset(X_train, y_train)
#        num_workers=4
        train_data = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)#,num_workers=num_workers)
        #X_test=nd.array(X_te).as_in_context(ctx)
        #y_test=nd.array(y_te).as_in_context(ctx)
        
    
        print('start training FC')
        FCnet=build_FC(train_data,ctx)    # initialize the overparametrized network
        FCnet.load_parameters('Selected_models/FCnet_'+str(datasetindex)+'_seed_'+str(seed),ctx=ctx)
        
        import torch.nn as nn
        model=fc(10)      
        loss = nn.MSELoss()
        dataset=torch.utils.data.TensorDataset(torch.Tensor(X_tr),torch.Tensor(y_tr))
        for i in range(6):
            model[int(i*2+1)].weight.data=torch.Tensor(FCnet[i].weight.data().asnumpy())
            model[int(i*2+1)].bias.data=torch.Tensor(FCnet[i].bias.data().asnumpy())
        
        
        print("dataset:",datasetindex,"seed",seed)
        print("before prune:",torch.sqrt(torch.mean((model(torch.Tensor(X_te))-torch.Tensor(y_te))**2))/factor)
        loss_before_prune.append(torch.sqrt(loss(model(torch.Tensor(X_te)),torch.Tensor(y_te)))/factor)  
        print(torch.sqrt(loss(model(torch.Tensor(X_te)),torch.Tensor(y_te)))/factor)
        # Prune ##

        device = torch.device("cpu")
        prune_loader = load.dataloader(dataset, 64, True, 4, 1)
        prune_epochs=10
        print('Pruning with {} for {} epochs.'.format('synflow', prune_epochs))
        pruner = load.pruner('synflow')(generator.masked_parameters(model, False, False, False))
        sparsity = 10**(-float(2.44715803134))  #280X    #100X 10**(-float(2))

        prune_loop(model, loss, pruner, prune_loader, device, sparsity, 
                  'exponential', 'global', prune_epochs, False, False, False, False)

        pruner.apply_mask()


        print("after prune:",torch.sqrt(loss(model(torch.Tensor(X_te)),torch.Tensor(y_te)))/factor)
        loss_after_prune.append(torch.sqrt(loss(model(torch.Tensor(X_te)),torch.Tensor(y_te)))/factor)
        ## post_train

        train_loader = load.dataloader(dataset, 64, True, 4)
        test_loader = load.dataloader(dataset, 200 , False, 4)
        optimizer = torch.optim.Adam(generator.parameters(model), betas=(0.9, 0.99))

        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[30,80], gamma=0.1)

        post_result = train_eval_loop(model, loss, optimizer, scheduler, train_loader, 
                                          test_loader, device, 100, True)   

        print("after post_train:",torch.sqrt(loss(model(torch.Tensor(X_te)),torch.Tensor(y_te)))/factor)
        loss_prune_posttrain.append(torch.sqrt(loss(model(torch.Tensor(X_te)),torch.Tensor(y_te)))/factor)

        num=0
        for i in pruner.masked_parameters:
            num=num+sum(sum(i[0]))
        print(num)
        NUM_PARA.append(num)
        seed=seed+1
        
        import mxnet.gluon.nn as nn



In [4]:
##synflow results 280X
a=0
for i in range(10):
    print(sum(loss_prune_posttrain[5*i:5*i+5])/5)
    a=a+sum(loss_prune_posttrain[5*i:5*i+5])/5
print("ave:",a/10)

tensor(0.0254, grad_fn=<DivBackward0>)
tensor(0.0483, grad_fn=<DivBackward0>)
tensor(0.0339, grad_fn=<DivBackward0>)
tensor(0.0349, grad_fn=<DivBackward0>)
tensor(0.0394, grad_fn=<DivBackward0>)
tensor(0.0314, grad_fn=<DivBackward0>)
tensor(0.0151, grad_fn=<DivBackward0>)
tensor(0.0179, grad_fn=<DivBackward0>)
tensor(0.0263, grad_fn=<DivBackward0>)
tensor(0.0218, grad_fn=<DivBackward0>)
ave: tensor(0.0294, grad_fn=<DivBackward0>)


In [148]:
##synflow results 100X
a=0
for i in range(10):
    print(sum(loss_prune_posttrain[5*i:5*i+5])/5)
    a=a+sum(loss_prune_posttrain[5*i:5*i+5])/5
print("ave:",a/10)

tensor(0.0260, grad_fn=<DivBackward0>)
tensor(0.0503, grad_fn=<DivBackward0>)
tensor(0.0372, grad_fn=<DivBackward0>)
tensor(0.0394, grad_fn=<DivBackward0>)
tensor(0.0383, grad_fn=<DivBackward0>)
tensor(0.0344, grad_fn=<DivBackward0>)
tensor(0.0150, grad_fn=<DivBackward0>)
tensor(0.0185, grad_fn=<DivBackward0>)
tensor(0.0275, grad_fn=<DivBackward0>)
tensor(0.0218, grad_fn=<DivBackward0>)
ave: tensor(0.0308, grad_fn=<DivBackward0>)
